# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

---
<h1>Содержание</h1>
<li><span><a href="#Открытие-и-изучение-файла-с-датасетом" data-toc-modified-id="Открытие-и-изучение-файла-с-датасетом-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Открытие и изучение файла с датасетом</a></span></li>
<li><span><a href="#Разбивка-данных-на-выборки" data-toc-modified-id="Разбивка-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбивка данных на выборки</a></span></li>
<li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span></li>
<li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li></ul></div>
<li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li>
<li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li>
<li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li>

---
## Открытие и изучение файла с датасетом
[К содержанию](#Содержание)

Сначала импортируем всё, что нам может понадобиться при рассчетах.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

rs = 12345 #зададим значение random_state один раз и на весь проект

Изучим исходные данные.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Всего 3214 объектов с 5 признаками. Так как нам нужно понять, кому из клиентов стоит порекомендовать тариф "Ультра", целевым признаком у нас будет являться значение столбца is_ultra. Так как вариантов столбца всего два - мы будем решать задачу бинарной классификации.

Предоставленные данные уже были предобработаны, в них отсутствуют пропуски. Типы данных можно было бы заменить (количество звонков "calls" и количество сообщений "messages" на целочисленные int, а is_ultra на булев bool), однако для нашей работы подойдут и данные имеющихся типов.

Посмотрим, сколько у нас пользователей тарифа "Ультра" в датасете:

In [3]:
len(df[df['is_ultra'] == 1])/len(df)

0.30647168637212197

Примерно 30.65%.

---
## Разбивка данных на выборки
[К содержанию](#Содержание)

Разобьем данные сначала на датасет с целевыми и прочими признаками. А затем разобьем эти датасеты на три части: учебную, валидационную и тестовую. Так как функции для разбивки на три части мне найти не удалось, просто применим разбивку на две части дважды.

Сначала разобьем датасет на две части в 20% и 80% от исходного датасета. Затем разобьем полученные 80% в пропорции 25% и 75%.  
Таким образом мы получим три выборки:

* учебную, составляющую 60% от исходного датасета
* валидационную, 20% от исходного датасета
* тестовую, 20% от исходного датасета

In [4]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

features_train_test, features_valid, target_train_test, target_valid = train_test_split(
    features,
    target,
    test_size=.20,
    random_state=rs)

features_train, features_test, target_train, target_test = train_test_split(
    features_train_test,
    target_train_test,
    test_size=.25,
    random_state=rs)

print('Размер обучающей выборки относительно всего датасета {:.1%} '.format(len(features_train)/len(df)))
print('Размер валидационной выборки относительно всего датасета {:.1%} '.format(len(features_valid)/len(df)))
print('Размер тестовой выборки относительно всего датасета {:.1%} '.format(len(features_test)/len(df)))

Размер обучающей выборки относительно всего датасета 60.0% 
Размер валидационной выборки относительно всего датасета 20.0% 
Размер тестовой выборки относительно всего датасета 20.0% 


---
## Исследование моделей
[К содержанию](#Содержание)

Исследуем каждую из трех моделей, изученных в курсе, подобрав в циклах оптимальные гипермараметры чтобы получить максимально возможное качество модели. Для определения качества мы будем использовать метрику <b>accuracy</b>, доля правильных ответов, данных моделью.

### Классификация древом решений

Начнем с классификации с помощью модели древа решений. Тут мы будем в цикле подбирать глубину max_depth. Если выбрать слишком малое значение, мождель получится недообученной, а если слишком высокое - переобученной. В цикле создадим и обучим модели с глубиной от 1 до 10 и сравним accuracy каждой из десяти. Выведем на экран все десять возможных вариантов с величиной их accuracy. Кроме того определим, при каком значении гиперпараметра модель получается наиболее точная.

In [5]:
accuracy_tree_best = 0
i_tree_best = 0
for i in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=rs, max_depth=i)
    model_tree.fit(features_train, target_train)
    train_predictions = model_tree.predict(features_valid)
    accuracy = accuracy_score(target_valid, train_predictions)
    print('При max_depth =', i, ' accuracy модели:', accuracy)
    if accuracy > accuracy_tree_best:
        accuracy_tree_best = accuracy
        i_tree_best = i
print()
print('Оптимальное значение max_depth =', i_tree_best, ' accuracy модели при этом:', accuracy_tree_best)

При max_depth = 1  accuracy модели: 0.7480559875583204
При max_depth = 2  accuracy модели: 0.7838258164852255
При max_depth = 3  accuracy модели: 0.7869362363919129
При max_depth = 4  accuracy модели: 0.7869362363919129
При max_depth = 5  accuracy модели: 0.7884914463452566
При max_depth = 6  accuracy модели: 0.7791601866251944
При max_depth = 7  accuracy модели: 0.7884914463452566
При max_depth = 8  accuracy модели: 0.7807153965785381
При max_depth = 9  accuracy модели: 0.7776049766718507
При max_depth = 10  accuracy модели: 0.7713841368584758

Оптимальное значение max_depth = 5  accuracy модели при этом: 0.7884914463452566


Максимально возможное accuracy классификации моделью древа решений 78.85% при глубине дерева в 5.

В работе я показываю проверку значений гиперпараметра до 10. Однако, я проверил, что получается при значении выше, до 50. Наибольшее accuracy модели при имеющихся исходных данных и значении random_state получается именно при глубине 5. Чтобы не перегружать работу, я не стал показывать этот перебор.

### Классификация моделью случайного леса

Далее исследуем классификацию моделью случайного леса. Тут мы будем подбирать два гиперпараметра: кроме глубины max_depth мы будем подбирать количество деревьев в лесу n_estimators. В цикле создадим и обучим модели с глубиной от 1 до 10 и с количеством деревьев от 10 до 50 с шагом в 10. Так как возможных комбинаций получается 50 штук, мы не будем выводить все на экран, а выведем только комбинацию с максимальным accuracy.

In [6]:
accuracy_forest_best = 0
i_forest_best = 0
es_forest_best = 0
for i in range(1, 11):
    for es in range(10, 51, 10):
        model_forest = RandomForestClassifier(random_state=rs, n_estimators=es, max_depth=i)
        model_forest.fit(features_train, target_train)
        train_predictions = model_forest.predict(features_valid)
        accuracy = accuracy_score(target_valid, train_predictions)
        #print('При max_depth =', i, ' и n_estimators=', es, ' accuracy модели:', accuracy)
        if accuracy > accuracy_forest_best:
            accuracy_forest_best = accuracy
            i_forest_best = i
            es_forest_best = es
print('Оптимальное значение max_depth =', i_forest_best, ' и n_estimators =', es_forest_best, ', accuracy модели при этом:', accuracy_forest_best)

Оптимальное значение max_depth = 6  и n_estimators = 20 , accuracy модели при этом: 0.8009331259720062


Максимальное accuracy модели случайного леса в 80.10% достигается при глубине 6 и количестве деревьев равном 20.

Я так же проверил accuracy модели при более широких диапазонах гиперпараметров. Существуют комбинации с более высоким accuracy (например, при n_estimators=100 и max_depth=9 accuracy модели составляет 80.40%), однако я посчитал, что десятые доли процента метрики не стоят того времени, которое требуется для их рассчета, поэтому в данном проекте я ограничился величинами, указанными в коде выше.

### Классификация моделью логистической регрессии

И наконец посмотрим accuracy при классификации моделью логистической регрессии. К сожалению, я не смог разобраться даже с помощью [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn-linear-model-logisticregression), как работает этот "черный ящик". У него есть ряд гиперпараметров, некоторые я пробовал изменять, но это не помогло мне улучшить accuracy модели. Видимо, для этого нужно глубже понимать, как работает сама модель. Так что я просто обучил модель и проанализировал результат.

In [7]:
model_logistic = LogisticRegression(random_state=rs, solver='lbfgs', max_iter=200)
model_logistic.fit(features_train, target_train)
train_predictions = model_logistic.predict(features_valid)
accuracy_logistic = accuracy_score(target_valid, train_predictions)
print('Accuracy при классификации моделью логистической регрессии:', accuracy_logistic)


Accuracy при классификации моделью логистической регрессии: 0.7589424572317263


Качество модели логистической регрессии равно 75.89%.

### Вывод из исследования моделей

Таким образом, проверив три модели, изученные в курсе, мы определили параметры каждой из них, при которых accuracy моделей на имеющихся данных максимально.

|Модель |Accuracy|Примечание|
|:-----|----:|:----|
|Древо решений |78.85% |При max_depth = 5|
|Случайный лес |80.10% |При max_depth = 6  и n_estimators = 20|
|Логистическая регрессия |75.89% |-|

На наших данных наилучшее значение метрики показала модель случайного леса, её мы и будем проверять на тестовой выборке.


---
## Проверьте модель на тестовой выборке
[К содержанию](#Содержание)

Возьмем выбранную модель с оптимальными параметрами и проверим её на тестовой выборке, созданной во втором разделе проекта.  
Используем гиперпараметры модели, при котор на наших учебных данных модель обучалась с максимальным значением accuracy.

In [8]:
model_test = RandomForestClassifier(random_state=rs, n_estimators=es_forest_best, max_depth=i_forest_best)
model_test.fit(features_train, target_train)
train_predictions_test = model_test.predict(features_test)
accuracy_test = accuracy_score(target_test, train_predictions_test)
print('Accuracy модели на тестовых данных', accuracy_test)        

Accuracy модели на тестовых данных 0.7853810264385692


На тестовом датасете наша модель демонстрирует accuracy 78.54%. Это приемлимо, согласно условий задания необходимо получить значение как минимум в 75%.

---
## Проверка модели на адекватность
[К содержанию](#Содержание)

In [16]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(features_train, target_train)

dummy_predictions = dummy.predict(features_test)
accuracy_dummy = accuracy_score(target_test, dummy_predictions)
print('Accuracy константной модели на тестовых данных', accuracy_dummy) 

Accuracy константной модели на тестовых данных 0.6889580093312597


Accuracy модели, обученной нами выше accuracy константной модели, а следовательно наша модель адекватна.

___

## Общий вывод
[К содержанию](#Содержание)

В ходе работы мы ознакомились с данными, проанализировали их и построили на их основе несколько моделей, подбирая наиболее эффективную на данном датасете.

В итоге лучше всего себя показала модель случайного леса, которую мы и протестировали на специально выделенной части данных.

Полученная модель на тестовых данных показала accuracy в 78.54%, что лучше минимально необходимого значения в 75%.

Это значит, что обученная модель пригодна для определения подходящего пользователю тарифа.

Кроме того, модель была проверена на адекватность путем сравнения accuracy модели со значением той же метрики для константной модели. Accuracy константной модели оказалось ниже, что значит наша модель адекватна.